In [1]:
import traitlets
import os
from jetbot import Camera, bgr8_to_jpeg
import ipywidgets.widgets as widgets
from IPython.display import display
from uuid import uuid1
import json

In [2]:
camera = Camera.instance(width=224, height=224)

image_widget = widgets.Image(format='jpeg', width=300, height=300)

traitlets.dlink((camera, 'value'), (image_widget, 'value'), transform=bgr8_to_jpeg)

display(image_widget)

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

In [7]:
controller = widgets.Controller(index=1)

display(controller)

Controller(index=1)

In [6]:
DATASET_DIR = 'dataset'
IMAGES_DIR = os.path.join(DATASET_DIR, 'images')
LABELS_DIR = os.path.join(DATASET_DIR, 'labels')

# we have this "try/except" statement because these next functions can throw an error if the directories exist already
try:
    os.makedirs(IMAGES_DIR)
    os.makedirs(LABELS_DIR)
except FileExistsError:
    print('Directories not created becasue they already exist')

In [8]:
for b in controller.buttons:
    b.unobserve_all()
for a in controller.axes:
    a.unobserve_all()

# get controller buttons / axes
save_button = controller.buttons[5]
left_button = controller.buttons[14]
forward_button = controller.buttons[12]
right_button = controller.buttons[15]
steering_axis = controller.axes[2]

# attach steering axis to widget since it's easier to visualize horizontally
steering_widget = widgets.FloatSlider(min=-1.0, max=1.0, step=0.001, description='steering')
widgets.jslink((steering_axis, 'value'), (steering_widget, 'value'))

# create widgets that will display the annotation to be saved
snapshot_widget = widgets.Image(format='jpeg', width=300, height=300)
left_steering = widgets.FloatSlider(min=-1.0, max=1.0, step=0.001, description='left steering')
forward_steering = widgets.FloatSlider(min=-1.0, max=1.0, step=0.001, description='forward steering')
right_steering = widgets.FloatSlider(min=-1.0, max=1.0, step=0.001, description='right steering')

def get_count():
    return len(os.listdir(LABELS_DIR))


count_widget = widgets.IntText(description='count', value=get_count())


def set_left(c):
    if c['new']:
        left_steering.value = steering_widget.value

def set_right(c):
    if c['new']:
        right_steering.value = steering_widget.value

def set_forward(c):
    if c['new']:
        forward_steering.value = steering_widget.value
    
def save(c):
    if c['new']:
        snapshot_widget.value = image_widget.value
        
        uuid = str(uuid1())
        image_name = uuid + '.jpg'
        image_path = os.path.join(IMAGES_DIR, image_name)
        label_path = os.path.join(LABELS_DIR, uuid + '.json')

        with open(image_path, 'wb') as f:
            f.write(snapshot_widget.value)

        label = {
            'image': image_name,
            'left_steering': left_steering.value,
            'forward_steering': forward_steering.value,
            'right_steering': right_steering.value
        }

        with open(label_path, 'w') as f:
            json.dump(label, f)

        count_widget.value = get_count()
        

left_button.observe(set_left, names='value')
forward_button.observe(set_forward, names='value')
right_button.observe(set_right, names='value')
save_button.observe(save, names='value')


display(
  widgets.HBox([image_widget, snapshot_widget]),
  widgets.VBox([steering_widget, left_steering, forward_steering, right_steering]),
  count_widget
)

IntText(value=63, description='count')

In [9]:
!zip -r -q lego_city.zip dataset